In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# model.to(device)

cuda


In [2]:

import torch

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Current GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA available: True
Number of GPUs: 1
Current GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# Define transformations (preprocessing)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50 expects 224x224
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale MRI to 3-channel
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

# Load datasets (assuming folder structure: train/top, train/side)
train_dataset = datasets.ImageFolder(root='C:\Projects Datasets\\filter_top_view\\train', transform=transform)  # Replace 'train' with your path
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load pre-trained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Freeze early layers (optional, for faster training)
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer for binary classification (top vs side)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),  # Binary output
    nn.Sigmoid()
)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Only train the new layers

# Training loop (adjust epochs as needed)
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)  # Binary labels (0=side, 1=top)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

# Save the model
torch.save(model.state_dict(), 'mri_top_view_classifier.pth')
print("Model trained and saved.")

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\chanu/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:10<00:00, 9.97MB/s]


Epoch 1/10, Loss: 0.5516
Epoch 2/10, Loss: 0.2951
Epoch 3/10, Loss: 0.1836
Epoch 4/10, Loss: 0.1347
Epoch 5/10, Loss: 0.1149
Epoch 6/10, Loss: 0.0994
Epoch 7/10, Loss: 0.0855
Epoch 8/10, Loss: 0.0736
Epoch 9/10, Loss: 0.0660
Epoch 10/10, Loss: 0.0657
Model trained and saved.


In [4]:
import torch
from torchvision import transforms, models
from PIL import Image
import os
import shutil

# Load the trained model
model = models.resnet50(weights=None)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)
model.load_state_dict(torch.load('mri_top_view_classifier.pth'))
model.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Preprocessing for inference
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to classify a single image
def is_top_view(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
    prob = output.item()
    return prob > 0.5  # Threshold; adjust if needed (e.g., for higher confidence)

# Filter the dataset
input_dir = 'C:\Projects Datasets\Segmentation_1\Segmentation\images'  # Your mixed dataset folder
output_dir = 'C:\Projects Datasets\\filter_top_view\processed'  # Where to save top views
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.nii', '.dcm')):  # Add MRI formats if needed
        image_path = os.path.join(input_dir, filename)
        if is_top_view(image_path):
            shutil.copy(image_path, os.path.join(output_dir, filename))

print(f"Filtered top views saved to {output_dir}")

C:\Users\chanu\AppData\Local\Temp\ipykernel_17168\1935598428.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('mri_top_view_classifier.p

Filtered top views saved to C:\Projects Datasets\filter_top_view\processed
